# Handle Nominal and Ordinal categorical dataset

## 1. Using _Z-score_
### Approach:
#### Replace to each corresponding $Z-score$ (element-wise)
$
\begin{align}
    Z = \frac{x - \mu} {\sigma} 
\end{align}
$


In [2]:
import pandas as pd
from scipy.stats import zscore

In [12]:
dataset_1 = pd.read_csv("dataset/auto-mpg.csv", na_values=['NA','?'])
dataset_1.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [13]:
# Apply Z-score elementwise
dataset_1['mpg'] = zscore(dataset_1['mpg'])
dataset_1.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,-0.706439,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,-1.090751,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,-0.706439,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,-0.962647,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,-0.834543,8,302.0,140.0,3449,10.5,70,1,ford torino


## 2. OneHotEncode a.k.a Dummies Variable(under Pandas)

In [14]:
dataset_2 = pd.read_csv("dataset/simple-dataset.csv", na_values=['NA','?'])
dataset_2.head()

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [15]:
areas = list(dataset_2['area'].unique())
print(f'Number of areas: {len(areas)}')
print(f'Areas: {areas}')

# Encode 
dummies = pd.get_dummies(dataset_2['area'], prefix='area')
# Concatenate
dataset_2 = pd.concat([dataset_2,dummies], axis=1)
# Drop OneHotEncoded feature
dataset_2.drop('area', axis=1, inplace=True)

dataset_2.head()

Number of areas: 4
Areas: ['c', 'd', 'a', 'b']


,id,job,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product,area_a,area_b,area_c,area_d
0,1,vv,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b,0,0,1,0
1,2,kd,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c,0,0,1,0
2,3,pe,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b,0,0,1,0
3,4,11,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b,0,0,1,0
4,5,kl,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a,0,0,0,1


## **3. Target encoding using `target-mean-group` without smoothing**
### Approach:
#### Replace label to the mean of its corresponding class values. 
#### Notice: it has a limitation when having an unbalanced label(case of `cat_1`)

In [4]:
# Create a small sample dataset
import pandas as pd
import numpy as np

np.random.seed(43) # seeding param

dataset_3 = pd.DataFrame({
    'cont_9': np.random.rand(10)*100,
    'cat_0': ['dog'] * 5 + ['cat'] * 5,
    'cat_1': ['wolf'] * 9 + ['tiger'] * 1,
    'y': [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
})
dataset_3

,cont_9,cat_0,cat_1,y
0,11.505457,dog,wolf,1
1,60.906654,dog,wolf,0
2,13.339096,dog,wolf,1
3,24.058962,dog,wolf,1
4,32.713906,dog,wolf,1
5,85.913749,cat,wolf,1
6,66.609021,cat,wolf,0
7,54.116221,cat,wolf,0
8,2.901382,cat,wolf,0
9,73.374830,cat,tiger,0


In [5]:
# Group by Label 
# Compute class mean for each group 
mean_by_group = dataset_3.groupby('cat_0')['y'].mean()
mean_by_group = dict(mean_by_group)

# Mean can be applied to substitute the categorical value 
# Map mean group to Label, 
# Drop categorical feature
dataset_3["cat_0_enc"] = dataset_3["cat_0"].map(mean_by_group)
dataset_3.drop('cat_0', axis=1, inplace=True)
dataset_3

,cont_9,cat_1,y,cat_0_enc
0,11.505457,wolf,1,0.8
1,60.906654,wolf,0,0.8
2,13.339096,wolf,1,0.8
3,24.058962,wolf,1,0.8
4,32.713906,wolf,1,0.8
5,85.913749,wolf,1,0.2
6,66.609021,wolf,0,0.2
7,54.116221,wolf,0,0.2
8,2.901382,wolf,0,0.2
9,73.374830,tiger,0,0.2


## 4. Target encoding using `target-mean-group` with smoothing (Using weight/smoothing factor $K$, here 5)
### Approach:
#### Smoothing the mean for each group category w.r.t `count frequency of labels`, `mean for each group category` and a `pre-defined constant weight K`. Here $K = 5$
#### Notice: 
#### 1. it has a limitation as well. It may overfit the model, so be cautious :)
#### 2. `df1` and `df2` are training and testing set respectively  
### Further reading: [kaggle](https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features)

In [18]:
# Source: https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
def calc_smooth_mean(df1, df2, cat_name, target, weight):
    # Compute the global mean
    mean = dataset[target].mean()

    # Compute the number of values and the mean of each group
    agg = dataset.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth), df2[cat_name].map(smooth.to_dict())

In [19]:
# same as dataset_3 
dataset_4 = pd.DataFrame({
    'cont_9': np.random.rand(10)*100,
    'cat_0': ['dog'] * 5 + ['cat'] * 5,
    'cat_1': ['wolf'] * 9 + ['tiger'] * 1,
    'y': [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
})
dataset_4 

,cont_9,cat_0,cat_1,y
0,39.495002,dog,wolf,1
1,80.204712,dog,wolf,0
2,25.442113,dog,wolf,1
3,5.688494,dog,wolf,1
4,86.664864,dog,wolf,1
5,22.102900,cat,wolf,1
6,40.498945,cat,wolf,0
7,31.609647,cat,wolf,0
8,7.666270,cat,wolf,0
9,84.322469,cat,tiger,0


In [21]:
# Weight factor: Adjusted upon the dataset 
WEIGHT = 5
dataset_4['cat_0_enc'] = calc_smooth_mean(df1=dataset_4, df2=None, cat_name='cat_0', target='y', weight=WEIGHT)
dataset_4['cat_1_enc'] = calc_smooth_mean(df1=dataset_4, df2=None, cat_name='cat_1', target='y', weight=WEIGHT)

dataset_4

# You can drop now :P

,cont_9,cat_0,cat_1,y,cat_0_enc,cat_1_enc
0,39.495002,dog,wolf,1,0.65,0.535714
1,80.204712,dog,wolf,0,0.65,0.535714
2,25.442113,dog,wolf,1,0.65,0.535714
3,5.688494,dog,wolf,1,0.65,0.535714
4,86.664864,dog,wolf,1,0.65,0.535714
5,22.102900,cat,wolf,1,0.35,0.535714
6,40.498945,cat,wolf,0,0.35,0.535714
7,31.609647,cat,wolf,0,0.35,0.535714
8,7.666270,cat,wolf,0,0.35,0.535714
9,84.322469,cat,tiger,0,0.35,0.416667
